In [1]:
from datasets import load_metric
metric = load_metric('glue', 'stsb')
metric

Using the latest cached version of the module from C:\Users\Administrator\.cache\huggingface\modules\datasets_modules\metrics\glue\479ff3f18b7b4e5a529ea7117f3ce02d081a7219ee662384b5633d830a11b981 (last modified on Wed Aug 24 18:12:12 2022) since it couldn't be found locally at D:\information\code\pycharmProjects\kaggle_learning\专利短语相似度\glue\glue.py, or remotely (ConnectionError).


Metric(name: "glue", features: {'predictions': Value(dtype='float32', id=None), 'references': Value(dtype='float32', id=None)}, usage: """
Compute GLUE evaluation metric associated to each GLUE dataset.
Args:
    predictions: list of predictions to score.
        Each translation should be tokenized into a list of tokens.
    references: list of lists of references for each translation.
        Each reference should be tokenized into a list of tokens.
Returns: depending on the GLUE subset, one or several of:
    "accuracy": Accuracy
    "f1": F1 score
    "pearson": Pearson Correlation
    "spearmanr": Spearman Correlation
    "matthews_correlation": Matthew Correlation
Examples:

    >>> glue_metric = datasets.load_metric('glue', 'sst2')  # 'sst2' or any of ["mnli", "mnli_mismatched", "mnli_matched", "qnli", "rte", "wnli", "hans"]
    >>> references = [0, 1]
    >>> predictions = [0, 1]
    >>> results = glue_metric.compute(predictions=predictions, references=references)
    >>> print

## 数据预处理

In [3]:
from transformers import AutoTokenizer
model_checkpoint = 'microsoft/deberta-v3-small'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
tokenizer("hellow, this one sentence!", "and this is also a sentence.")

{'input_ids': [1, 12018, 2985, 261, 291, 311, 4378, 300, 2, 263, 291, 269, 327, 266, 4378, 260, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [5]:
import pandas as pd
input_path = 'data/uspatent/'
df = pd.read_csv(input_path+'train.csv')
df_title = pd.read_csv(input_path+'titles.csv')

In [6]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [7]:
df_title

,code,title,section,class,subclass,group,main_group
0,A,HUMAN NECESSITIES,A,NaN,NaN,NaN,NaN
1,A01,AGRICULTURE; FORESTRY; ANIMAL HUSBANDRY; HUNTI...,A,1.0,NaN,NaN,NaN
2,A01B,SOIL WORKING IN AGRICULTURE OR FORESTRY; PARTS...,A,1.0,B,NaN,NaN
3,A01B1/00,Hand tools (edge trimmers for lawns A01G3/06 ...,A,1.0,B,1.0,0.0
4,A01B1/02,Spades; Shovels {(hand-operated dredgers E02F3...,A,1.0,B,1.0,2.0
...,...,...,...,...,...,...,...
260471,Y10T483/1864,including tool pot or adapter,Y,10.0,T,483.0,1864.0
260472,Y10T483/1873,Indexing matrix,Y,10.0,T,483.0,1873.0
260473,Y10T483/1882,Rotary disc,Y,10.0,T,483.0,1882.0
260474,Y10T483/1891,Chain or belt,Y,10.0,T,483.0,1891.0


In [8]:
df_title[['code', 'class']]['9022']

KeyError: '9022'

In [9]:
df.columns, df_title.columns

(Index(['id', 'anchor', 'target', 'context', 'score'], dtype='object'),
 Index(['code', 'title', 'section', 'class', 'subclass', 'group', 'main_group'], dtype='object'))

In [10]:
df = df.merge(df_title, how='left', left_on='context', right_on='code')

In [11]:
df.columns, df_title.columns

(Index(['id', 'anchor', 'target', 'context', 'score', 'code', 'title',
        'section', 'class', 'subclass', 'group', 'main_group'],
       dtype='object'),
 Index(['code', 'title', 'section', 'class', 'subclass', 'group', 'main_group'], dtype='object'))

In [12]:
df = df[['id', 'anchor', 'target', 'context', 'score', 'title']]

In [13]:
df

,id,anchor,target,context,score,title
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...
...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,DECORATIVE ARTS
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,DECORATIVE ARTS
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,DECORATIVE ARTS
36471,756ec035e694722b,wood article,wooden material,B44,0.75,DECORATIVE ARTS


In [14]:
from sklearn.model_selection import StratifiedGroupKFold
kf = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=42)
df['fold']=-1


In [20]:
df['anchor']

0           abatement
1           abatement
2           abatement
3           abatement
4           abatement
             ...     
36468    wood article
36469    wood article
36470    wood article
36471    wood article
36472    wood article
Name: anchor, Length: 36473, dtype: object

## 这里到吗不明白

In [21]:
for  f, (t_, v_) in enumerate(kf.split(X=df, y=df['anchor'], groups=df['anchor'])):
    # print('f', f)
    # print('t', t_)
    # print('v', v_)
    df.loc[v_, 'fold'] = f

D:\ProgramData\Anaconda3\envs\pytorch_env\lib\site-packages\sklearn\model_selection\_split.py:880: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [22]:
df

,id,anchor,target,context,score,title,fold
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,4
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,4
2,36d72442aefd8232,abatement,active catalyst,A47,0.25,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,4
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,4
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00,FURNITURE; DOMESTIC ARTICLES OR APPLIANCES; CO...,4
...,...,...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00,DECORATIVE ARTS,2
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50,DECORATIVE ARTS,2
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50,DECORATIVE ARTS,2
36471,756ec035e694722b,wood article,wooden material,B44,0.75,DECORATIVE ARTS,2


In [24]:
df['input'] = df['anchor'] + tokenizer.sep_token + df['title'].apply(str.lower)

In [26]:
import torch
from torch.utils.data import DataLoader, Dataset
class TrainDataset(Dataset):
    def __init__(self, df):
        self.inputs = df['input'].values.astype(str)
        # 这个target要研究一下 题目要求 不知道作用
        self.targets = df['target'].values.astype(str)
        self.label = df['score'].values

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, item):
        inputs = self.inputs[item]
        targets = self.targets[item]
        label = self.label[item]

        inputs = tokenizer(
            inputs,
            targets,
            padding = 'max_length',
            truncation=True
                           )
        return {**inputs,
                'label':torch.as_tensor(label, dtype=torch.float)}

## 模型微调
### 这个numberlayer不明白 官方文档也没有

In [31]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 1
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint,num_labels=num_labels)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.dense.weight', 'mask_predictions.LayerNorm.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [34]:
metric_name = 'person'
model_name = model_checkpoint.split("/")[-1]
batch_size = 200
args = TrainingArguments(
    f"{model_name}-finetuned",
    evaluation_strategy='epoch',
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    save_total_limit=1,
)

In [35]:
def computer_metrics(eval_pred):
    predictions, labels = eval_pred
    return metric.compute(predictions=predictions,references = labels)

In [36]:
train_dataset = TrainDataset(df[df['fold']!=0])
val_dataset = TrainDataset(df[df['fold']==0])
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=computer_metrics,
)

In [37]:
trainer.train()

D:\ProgramData\Anaconda3\envs\pytorch_env\lib\site-packages\transformers\optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 29429
  Num Epochs = 5
  Instantaneous batch size per device = 200
  Total train batch size (w. parallel, distributed & accumulation) = 200
  Gradient Accumulation steps = 1
  Total optimization steps = 740
Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: CUDA out of memory. Tried to allocate 300.00 MiB (GPU 0; 2.00 GiB total capacity; 1.05 GiB already allocated; 0 bytes free; 1.08 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF